In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf

import pip
pip.main(['install', 'xgboost'])
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
import matplotlib.pyplot as plt

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
bat_charge_path = r"/home/yamukelani/source_code/msc_research/rul_prediction/src/datasets/all_batteries_charging_data.csv"
bat_discharge_path = r"/home/yamukelani/source_code/msc_research/rul_prediction/src/datasets/all_batteries_discharging_data.csv"

battery_charging = pd.read_csv(bat_charge_path)
battery_discharging = pd.read_csv(bat_discharge_path)

In [3]:
bat_discharge_sel = battery_discharging[['Ambient_temperature', 'Voltage_measured',
       'Current_measured', 'Temperature_measured', 'Current_load',
       'Voltage_load', 'Discharge_time', 'Capacity']].set_index('Discharge_time')
bat_discharge_sel


,Ambient_temperature,Voltage_measured,Current_measured,Temperature_measured,Current_load,Voltage_load,Capacity
Discharge_time,,,,,,,
0.000,24,4.191492,-0.004902,24.330034,-0.0006,0.000,1.856487
16.781,24,4.190749,-0.001478,24.325993,-0.0006,4.206,1.856487
35.703,24,3.974871,-2.012528,24.389085,-1.9982,3.062,1.856487
53.781,24,3.951717,-2.013979,24.544752,-1.9982,3.030,1.856487
71.922,24,3.934352,-2.011144,24.731385,-1.9982,3.011,1.856487
...,...,...,...,...,...,...,...
2686.359,24,3.443760,-0.002426,35.383979,0.0006,0.000,1.341051
2700.546,24,3.453271,-0.000981,35.179732,0.0006,0.000,1.341051
2714.640,24,3.461963,0.000209,34.977000,0.0006,0.000,1.341051


# Split the battery discharging data into training and test datasets. 

In [11]:
bat_discharge_train, bat_discharge_test = train_test_split(bat_discharge_sel, test_size=0.3)
X_train = bat_discharge_train[['Ambient_temperature', 'Voltage_measured', 'Current_measured', 'Temperature_measured', 'Current_load', 'Voltage_load']]
y_train = bat_discharge_train[['Capacity']]
X_test = bat_discharge_test[['Ambient_temperature', 'Voltage_measured', 'Current_measured', 'Temperature_measured', 'Current_load', 'Voltage_load']]
y_test = bat_discharge_test[['Capacity']]

In [12]:
bat_discharge_train.head(10)

,Ambient_temperature,Voltage_measured,Current_measured,Temperature_measured,Current_load,Voltage_load,Capacity
Discharge_time,,,,,,,
66.281,24,3.932439,-2.009598,24.130555,1.9988,3.021,1.797102
1106.000,24,3.550628,-2.009560,30.312309,1.9990,2.645,1.603524
2974.985,24,3.562115,-0.002523,35.876573,0.0006,0.000,1.449042
1574.516,24,3.357780,-2.008311,34.542038,1.9990,2.446,1.420701
498.125,24,3.763987,-2.006470,27.554253,1.9986,2.859,1.648224
75.828,24,3.836312,-2.011151,25.537846,1.9988,2.918,1.378719
2454.781,24,3.439623,-2.010389,33.994183,1.9982,2.490,1.751730
1031.453,24,3.523641,-2.011124,30.426283,1.9988,2.618,1.509392
2154.640,24,3.490675,-2.009732,33.315500,-1.9990,2.596,1.867570


In [13]:
bat_discharge_test.head(10)

,Ambient_temperature,Voltage_measured,Current_measured,Temperature_measured,Current_load,Voltage_load,Capacity
Discharge_time,,,,,,,
3409.656,24,3.223324,-0.001598,36.768178,-0.0006,0.000,1.813440
2689.906,24,3.371231,-2.012041,34.230154,1.9990,2.469,1.771013
1584.328,24,3.561380,-1.989713,31.143738,2.0000,2.626,1.806264
927.953,24,3.519920,-2.013802,31.054653,1.9980,2.539,1.303410
338.047,24,3.750069,-2.010790,28.268624,1.9980,2.776,1.349315
113.328,24,3.905826,-1.989371,25.592611,2.0000,2.955,1.565281
1817.969,24,3.365431,-2.010967,33.926124,1.9990,2.459,1.503997
2165.562,24,3.401571,-0.000521,40.487310,0.0006,0.000,1.153818
328.641,24,3.794609,-2.007364,26.872865,1.9988,2.883,1.660759


In [14]:
len(bat_discharge_test)/len(bat_discharge_sel)

0.300003769094502

In [15]:
len(bat_discharge_train)/len(bat_discharge_sel)

0.699996230905498

In [16]:
len(bat_discharge_sel)

185721

In [17]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import mean_squared_error, r2_score

# Choose Kernel function

In [18]:
# Choose kernel function
kernel = ConstantKernel(constant_value=1.0) * RBF(length_scale=1.0) # Product of constant and RBF kernels

# Create and fit model

In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_s=scaler.fit_transform(X_train)
X_test_s=scaler.fit_transform(X_test)

In [21]:
X_test_s

array([[0.        , 0.58835544, 0.99313855, 0.72150407, 0.49985   ,
        0.        ],
       [0.        , 0.64928978, 0.007344  , 0.59447574, 0.99975   ,
        0.58258613],
       [0.        , 0.72762704, 0.01829197, 0.44000033, 1.        ,
        0.6196319 ],
       ...,
       [0.        , 0.68372569, 0.01839705, 0.59424337, 1.        ,
        0.5946201 ],
       [0.        , 0.72043599, 0.00805048, 0.39833277, 0.99975   ,
        0.62340727],
       [0.        , 0.78349854, 0.00747313, 0.37494784, 0.99955   ,
        0.64157622]])

In [ ]:
# Create and fit model
# Create GaussianProcessRegressor object with kernel and noise variance
gpr_model = GaussianProcessRegressor(kernel=kernel, alpha=noise**2) 
gpr_model.fit(X_train, y_train) # Fit model to training data

In [33]:
len(X_train)

404

In [35]:
type(X)

numpy.ndarray